In [1]:
from collections import defaultdict
def read_and_process_file(filename='input'):
    file1 = open(f'{filename}_19', 'r')
    blueprints = {}
    for l in file1.readlines():
        row = l.split(" ")
        blueprint_id = int(row[1][:-1])
        scenario = {}
        scenario[0] = [int(row[6]),0,0,0]
        scenario[1] = [int(row[12]),0,0,0]
        scenario[2] = [int(row[18]), int(row[21]),0,0]
        scenario[3] = [int(row[27]), 0, int(row[30]),0]
        blueprints[blueprint_id] = scenario
    return blueprints

In [2]:
def check_if_possile(robot_id, scenario, robots):
    return all([robots[i]>0 for i,req in enumerate(scenario[robot_id]) if req>0])

In [84]:
#CURRENT
def optimize_blueprint(scenario, time):
    
    starting_path = ([1,0,0,0], [0,0,0,0], None, time)
    paths = [starting_path]
    best = 0
    counter = defaultdict(int)
    best_path = None
    max_requirements = {_id:max([req[_id] for _, req in scenario.items()]) for _id in scenario.keys()}
    
    while(paths):
        _robots, _resources , _previous_id, _time = paths.pop()
        show = False
        if _time == 0:
            if _resources[3] > best:
                best_path = (_robots, _resources)
            best = max(best, _resources[3])
            continue
        
        ## - cannot beat max geode   
        if  (_resources[3] + (_robots[3]*_time) + (_time*(_time - 1)//2)) <= best:
            counter['cannot beat best result'] += 1
            if show:
                print("Cannot beat")
            continue
        
        #explore all options
        wait = False
        for robot_id in range(4): 
            ## - not enough resources to build it
            if any([_resources[i]<req for i,req in enumerate(scenario[robot_id])]):
                if check_if_possile(robot_id, scenario, _robots):
                    wait = True
                counter['wait'] += 1
                if show:
                    print("Wait")
                continue

                
            ## - we have more than enough resources (can build any day until finish) - no point in builidng
            if robot_id != 3:
                if (max_requirements[robot_id]*_time) < _resources[robot_id] + _robots[robot_id]*_time:
                    counter['too many resources already'] += 1
                    if show:
                        print("Too many resources already")
                    continue

                
            ## - previously waited - skip the ones that could have been build
            if _previous_id is None:
                if all([(_resources[i]-_robots[i])>=req for i,req in enumerate(scenario[robot_id])]):
                    if show:
                        print("Previously built")
                    counter['previously could have been built'] += 1
                    continue
            
            _new_robots = [r+int(i==robot_id) for i,r in enumerate(_robots)]
            _new_resources = [r+_robots[i]-scenario[robot_id][i] for i,r in enumerate(_resources)]
            paths.append((_new_robots, _new_resources, robot_id, _time-1))
        if wait:
            _new_resources = [r+_robots[i] for i,r in enumerate(_resources)]
            paths.append((_robots.copy(), _new_resources, None, _time-1))
    return best, counter, best_path

#### One star

In [94]:
data = read_and_process_file('input')
res = 0
_debug = {}
for _id in range(len(data)):
    time = 24
    _debug[_id] = optimize_blueprint(data[_id+1], time)[0]
    res += (_id+1)*_debug[_id]
print(f"The result is {res}")

The result is 1480


#### Two stars

In [100]:
data = read_and_process_file('input')
res = 1
_debug = {}
for _id in range(3):
    time = 32
    _debug[_id] = optimize_blueprint(data[_id+1], time)[0]
    res *= _debug[_id]
print(f"The result is {res}")

The result is 3168


In [ ]:
#Never finishes - all options possible (cross product)
def optimize_blueprint(scenario, time, robots = [1,0,0,0], in_progress=[0,0,0,0], state = [0,0,0,0]):
    if time<1:
        return state[3]
    if in_progress[0]==1:
        if state[0] < scenario['ore']:
            return -1
        else:
            state[0] -= scenario['ore']
            
    if in_progress[1]==1:
        if state[0] < scenario['clay']:
            return -1
        else:
            state[0] -= scenario['clay']
            
    if in_progress[2]==1:
        if (state[0] < scenario['obisidian'][0]) | (state[1] < scenario['obisidian'][1]):
            return -1
        else:
            state[0] -= scenario['obisidian'][0]
            state[1] -= scenario['obisidian'][1]
            
    if in_progress[3]==1:
        if (state[0] < scenario['geode'][0]) | (state[2] < scenario['geode'][1]):
            return -1
        else:
            state[0] -= scenario['geode'][0]
            state[2] -= scenario['geode'][1]
    
    #collect and finish building
    for i in range(4):
        state[i] += robots[i]
        robots[i] += in_progress[i]
    
    return max(
        optimize_blueprint(scenario, time-1, robots.copy(), [0,0,0,0], state.copy()),   #wait
        optimize_blueprint(scenario, time-1, robots.copy(), [1,0,0,0], state.copy()),   #create ore
        optimize_blueprint(scenario, time-1, robots.copy(), [0,1,0,0], state.copy()),   #create clay
        optimize_blueprint(scenario, time-1, robots.copy(), [0,0,1,0], state.copy()),   #create obsidian
        optimize_blueprint(scenario, time-1, robots.copy(), [0,0,0,1], state.copy())    #create geode
    )

In [ ]:
data = read_and_process_file('test')
time = 13
res = optimize_blueprint(data[1], time)
res

In [ ]:
#Too naive - doesnt find optimum - build the same type or next (no coming back)
def optimize_blueprint(scenario, time, robots = [1,0,0,0], state = [0,0,0,0]):
    if time<1:
        return state[3]
    res = []
    print()
    print(f"Time: {25-time}")
    if (state[0] >= scenario['geode'][0]) and (state[2] >= scenario['geode'][1]):
        print("-> Geode")
        state[0] -= scenario['geode'][0]
        state[2] -= scenario['geode'][1]
        for i in range(4):
            state[i] += robots[i]
        robots[3] += 1
        
        print(f"Robots: {robots}")
        print(f"Assets: {state}")
        
        res.append(optimize_blueprint(scenario, time-1, robots.copy(), state.copy()))
    elif (state[0] < scenario['geode'][0]) and (state[2] >= scenario['geode'][1]):
        print("-> Wait for Geode")
        for i in range(4):
            state[i] += robots[i]
        
        print(f"Robots: {robots}")
        print(f"Assets: {state}")
        
        res.append(optimize_blueprint(scenario, time-1, robots.copy(), state.copy()))
    elif (state[0] >= scenario['obisidian'][0]) and (state[1] >= scenario['obisidian'][1]):
        print("-> Obsidian")
        state[0] -= scenario['obisidian'][0]
        state[1] -= scenario['obisidian'][1]
        for i in range(4):
            state[i] += robots[i]
        robots[2] += 1
            
        print(f"Robots: {robots}")
        print(f"Assets: {state}")
        res.append(optimize_blueprint(scenario, time-1, robots.copy(), state.copy()))
    elif (state[0] < scenario['obisidian'][0]) and (state[1] >= scenario['obisidian'][1]):
        print("-> Wait for Obsidian")

        for i in range(4):
            state[i] += robots[i]
            
        print(f"Robots: {robots}")
        print(f"Assets: {state}")
        res.append(optimize_blueprint(scenario, time-1, robots.copy(), state.copy()))
    elif state[0] >= scenario['clay']:
        print("-> Clay")
        state[0] -= scenario['clay']
        #collect and finish building
        for i in range(4):
            state[i] += robots[i]
        robots[1] += 1

        print(f"Robots: {robots}")
        print(f"Assets: {state}")
        res.append(optimize_blueprint(scenario, time-1, robots.copy(), state.copy()))
    else:
        for i in range(4):
            state[i] += robots[i]

        print(f"Robots: {robots}")
        print(f"Assets: {state}")
        res.append(optimize_blueprint(scenario, time-1, robots.copy(), state.copy()))
    
    return max(res)

In [ ]:
def read_and_process_file(filename='input'):
    file1 = open(f'{filename}_19', 'r')
    blueprints = {}
    for l in file1.readlines():
        row = l.split(" ")
        blueprint_id = int(row[1][:-1])
        scenario = {}
        scenario["ore"] = int(row[6])
        scenario["clay"] = int(row[12])
        scenario["obisidian"] = (int(row[18]), int(row[21]))
        scenario["geode"] = (int(row[27]), int(row[30]))
        blueprints[blueprint_id] = scenario
    return blueprints

In [ ]:
def collect(robots, state):
    for i in range(4):
        state[i] += robots[i]
    return state

In [ ]:
def check_requirements(scenario, state, mode):
    new_state = state.copy()
    if mode == 0:
        new_state[0] -= scenario['ore']
    elif mode == 1:
        new_state[0] -= scenario['clay']
    elif mode == 2:
        new_state[0] -= scenario['obisidian'][0]
        new_state[1] -= scenario['obisidian'][1]
    else:
        new_state[0] -= scenario['geode'][0]
        new_state[2] -= scenario['geode'][1]
        
    if min(new_state) >= 0:
        return (True, new_state)
    else:
        return (False, state)

In [ ]:
## Too naive - eithjer build the same type or next type. One jump back possible
def optimize_blueprint(scenario, time, robots = [1,0,0,0], state = [0,0,0,0], mode=0, jump = False):
    if mode>3:
        return -1
    if time<1:
        return state[3]
    
    is_eligible, state = check_requirements(scenario, state, mode)
    state = collect(robots, state)
    
    if is_eligible:
        robots[mode] +=1
    
    if jump:
        return max(
        optimize_blueprint(scenario, time-1, robots.copy(), state.copy(), mode, jump),
        optimize_blueprint(scenario, time-1, robots.copy(), state.copy(), mode+1, jump),
        )
    else:
        return max(
            optimize_blueprint(scenario, time-1, robots.copy(), state.copy(), mode, jump),
            optimize_blueprint(scenario, time-1, robots.copy(), state.copy(), mode+1, jump),
            optimize_blueprint(scenario, time-1, robots.copy(), state.copy(), mode-1, True),
        )

In [ ]:
data = read_and_process_file('test')
_debug = {}
res = 0
for _id, scenario in data.items():
    time = 24
    _debug[_id] = optimize_blueprint(data[_id], time, [1,0,0,0], [0,0,0,0])
    res += _id*_debug[_id]
print(f"The result is {res}")